In [1]:
import pandas as pd
import numpy as np
import math
import json
import yaml
import operator

from itertools import combinations
from collections import Counter as ctr

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet

In [2]:
idf_df = pd.read_csv('idf.csv')

In [3]:
idf_df.head()

,Unnamed: 0,word,idf
0,0,cowhide,14.515398
1,1,porcupine,10.784031
2,2,untruthfully,15.964705
3,3,rebooting,14.141583
4,4,purcell,10.627598


In [4]:
highest_count_word_df = pd.read_csv('max_d.csv')
highest_count_word_df = highest_count_word_df.rename(columns={'Unnamed: 0': 'DocID'})

#highest_count_word_df[highest_count_word_df.DocID == 17809]
highest_count_word_df.head()

,DocID,max_d
0,0,11
1,1,3
2,2,2
3,3,7
4,4,5


In [5]:
df_index = pd.read_csv('index.csv', names=['word', 'frequency'])

In [6]:
df_index.head()

,word,frequency
0,cowhide,"{29104: 1, 82126: 1, 90797: 9, 108214: 1, 1287..."
1,porcupine,"{17809: 2, 19610: 1, 20467: 2, 25916: 12, 2815..."
2,untruthfully,"{53136: 1, 123616: 1, 128567: 3, 249592: 1, 25..."
3,rebooting,"{29971: 1, 58410: 1, 61198: 1, 64833: 1, 96158..."
4,purcell,"{570: 1, 5495: 2, 17002: 1, 17676: 1, 18439: 5..."


In [9]:
ws = ['this', 'is', 'a', 'test']

co = combinations(ws, len(ws)-1)

# for comb in co:
#     print(list(comb))


In [68]:
df_index[df_index['word']=='york']

,word,frequency
54064,york,"{3: 4, 7: 1, 10: 1, 28: 1, 32: 2, 66: 1, 111: ..."


In [7]:
lemmatizer = WordNetLemmatizer()
stops = stopwords.words('english')
coll_stops = ["also", "first", "one", "new", "year", "two", "time", "state", "school"]
stops.extend(coll_stops)

def clean(doc):
    doc_low = doc.lower().replace("–", " ")
    words = word_tokenize(doc_low)
    words = [lemmatizer.lemmatize(w).strip() for w in words if not w in stops and wordnet.synsets(w) and w.isalpha()]
    return words

In [8]:
#THERE HAS TO BE AT LEAST TWO NONSTOP LEMMATIZED WORDS IN THE QUERY FOR THIS TO WORK
#IDEALLY THERE IS 3+ WORDS
def get_candidate_resources(query):
    clean_query = clean(query)
    CR = []
    word_key_list = []
    if len(clean_query) == 1:
        only_word = clean_query[0]    #sw = single word
        sw_dict = yaml.load(df_index.loc[df_index.word==only_word, 'frequency'].item())
        CR.extend(list(sw_dict.keys()))
        if len(CR) > 50:
            return CR[:50]
        else: return CR
    for w in clean_query:
        w_dict = yaml.load(df_index.loc[df_index.word==w, 'frequency'].item())
        word_key_list.append(list(w_dict.keys()))
    CR.extend(set(word_key_list[0]).intersection(*word_key_list[1:]))
    if len(clean_query) == 2: #and len(CR) < 50:
        if len(CR)>50:
            return CR[:50]
        else: return CR
#         return CR
#         first_word = clean_query[0]
#         second_word = clean_query[1]
#         first_word_dict = yaml.load(df_index.loc[df_index.word==first_word, 'frequency'].item())
#         CR.extend(list(first_word_dict.keys()))
#         if len(set(CR))<50:
#             second_word_dict = yaml.load(df_index.loc[df_index.word==second_word, 'frequency'].item())
#             CR.extend(list(second_word_dict.keys()))
#             if len(set(CR)) > 50:
#                 return set(CR)[:49]
#             else: return set(CR)
#         else: return set(CR)
    if len(CR)<50:     #if the list of candidate resources is less than 50
        combs = combinations(clean_query, len(query)-1)     #use n-1 terms from the query
        for comb in list(combs):        #for combination in combinations
            word_key_list = []
            for w in list(comb):        #for word in n-1 combination
                w_dict = yaml.load(df_index.loc[df_index.word==w, 'frequency'].item())
                word_key_list.append(list(w_dict.keys()))
            CR.extend(set(word_key_list[0]).intersection(*word_key_list[1:])) #find the intersection between all n-1 query terms
            CR = set(CR)
            if len(set(CR))<50:
                continue
            elif len(set(CR))>50:
                CR = set(CR)[:50]
                break
            else: break
    return CR

In [9]:
def idf(w):
    idf_value = idf_df.loc[idf_df.word==w, 'idf'].item()
    return int(idf_value)

In [10]:
def frequency(w,d):
    word_freq_dict = yaml.load(df_index.loc[df_index.word==w, 'frequency'].item())
    word_freq = word_freq_dict[d]
    return int(word_freq)

In [11]:
def term_freq(w,d):
    max_showing_word = int(highest_count_word_df.loc[highest_count_word_df.DocID==d, 'max_d'].item())
    return frequency(w,d)/max_showing_word

In [12]:
# def relevance_ranking(q, cand_resources):
#     clean_query = clean(q)
#     rel_docs = {}
#     for d in cand_resources:
#         rel_score = 0
#         for word in clean_query:
#             result = term_freq(word,d) * idf(word)
#             rel_score += result
#         rel_docs[d] = rel_score
#     sorted_rel_docs = sorted(rel_docs.items(), key=operator.itemgetter(1), reverse=True)
#     results = [i[0] for i in sorted_rel_docs[:5]]
#     return results

In [13]:
def relevance_ranking(q, cand_resources):
    clean_query = clean(q)
    rel_docs = {}
    for d in cand_resources:
        rel_score = 0
        if len(clean_query)==1:
            w = clean_query[0]
            result = term_freq(w,d) * idf(w)
            rel_docs[d] = result
        else:
            for word in clean_query:
                result = term_freq(word,d) * idf(word)
                rel_score += result
            rel_docs[d] = rel_score
    sorted_rel_docs = sorted(rel_docs.items(), key=operator.itemgetter(1), reverse=True)
    results = [i[0] for i in sorted_rel_docs[:5]]
    return sorted_rel_docs

In [21]:
q = "beautiful dogs love"
r = get_candidate_resources(q)

In [22]:
#pass_to_query_suggestions = relevance_ranking(q, r)

In [23]:
#pass_to_query_suggestions